In [ ]:
import pm4py
from tokenreplay import TokenReplay
from comparison import ModelComparator
from practical.ProcessMining.group1.shared.utils import event_log_to_dataframe, read_txt_test_logs

In [ ]:
def compare(x_str="f", y_str="s", log_key="L17"):
    logs = read_txt_test_logs("../shared/example_files/simple_event_logs.txt")
    log = logs[log_key]
    log = event_log_to_dataframe(log)
    log = pm4py.format_dataframe(log, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
    
    net_alpha, im_alpha, fm_alpha = pm4py.discover_petri_net_alpha(log)
    net_heuristic, im_heuristic, fm_heuristic = pm4py.discover_petri_net_heuristics(log)
    net_inductive, im_inductive, fm_inductive = pm4py.discover_petri_net_inductive(log)
    
    m1 = TokenReplay(log, net_alpha, im_alpha, fm_alpha, "alpha")
    m2 = TokenReplay(log, net_heuristic, im_heuristic, fm_heuristic, "heuristic")
    m3 = TokenReplay(log, net_inductive, im_inductive, fm_inductive, "inductive")
    
    comparator = ModelComparator([m1, m2, m3])
    
    print("===== All models values =====")
    print(comparator.get_models_values())
    print("=============================\n")
    
    print("===== Pareto Efficient Models =====")
    results = comparator.run(x_dimension=x_str, y_dimension=y_str)
    print(results)
    print("=============================\n")

In [ ]:
compare()